In [0]:
Prepocess

In [1]:
from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
os.chdir("drive/My Drive/data")
!ls

test_tweets_unlabeled.txt  train_tweets  train_tweets.txt


In [1]:
import pandas as pd 
import numpy as np 
import scipy as sp 
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from sklearn.naive_bayes import MultinomialNB 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

# Text Processing

    Read each sentence from training file and remove the tabs and extract id and tweet seperately. 

In [4]:
file = "data/train_tweets.txt"
temp = []
with open(file, 'r',encoding="utf-8") as data:
    for line in data:
        row = []
        line = line.replace('\t'," ")
        elem = line.strip().split(" ")
        row.append(elem[0])
        row.append(" ".join(elem[1:]))
        temp.append(row) 

In [5]:
tw = pd.DataFrame(temp,columns = ["User","Tweet"])
temp = []
elem = []

In [6]:
tw.head()

,User,Tweet
0,8746,@handle Let's try and catch up live next week!
1,8746,Going to watch Grey's on the big screen - Thur...
2,8746,@handle My pleasure Patrick....hope you are well!
3,8746,@handle Hi there! Been traveling a lot and lot...
4,8746,RT @handle Looking to Drink Clean & Go Green? ...


In [7]:
#data pre-processing
tw['num_of_words'] = tw["Tweet"].str.split().apply(len)
tw.drop(tw[tw.num_of_words<4].index, inplace=True)
tw["Tweet"].replace(r"http\S+", "URL", regex=True,inplace=True)
tw["Tweet"].replace(r"@\S+", "REF", regex=True ,inplace=True)
tw["Tweet"].replace(r"(\d{1,2})[/.-](\d{1,2})[/.-](\d{2,4})+", "DATE", regex=True,inplace=True)
tw["Tweet"].replace(r"(\d{1,2})[/:](\d{2})[/:](\d{2})?(am|pm)+", "TIME", regex=True,inplace=True)
tw["Tweet"].replace(r"(\d{1,2})[/:](\d{2})?(am|pm)+", "TIME", regex=True,inplace=True)
tw["Tweet"].replace(r"\d+", "NUM", regex=True,inplace=True)
len(tw)

312494

In [120]:
cnt_user = tw['User'].value_counts()
cnt_user.describe()

count    9290.000000
mean       33.637675
std        27.250137
min         1.000000
25%        17.000000
50%        29.000000
75%        37.000000
max       276.000000
Name: User, dtype: float64

In [121]:
filtered_users=[]
Users=tw.User.unique().tolist()

for a in Users:
    if cnt_user[a]>=20:
      filtered_users.append(a)
    
print(len(filtered_users))    
  

5306


In [122]:
import random
from sklearn.model_selection import train_test_split
#20 random sample rows for each author
df_new=pd.DataFrame()
twts_train=pd.DataFrame()
twts_test=pd.DataFrame()
author_train=pd.DataFrame()
author_test=pd.DataFrame()

for a in filtered_users:
    rows = random.sample(list(tw[tw['User']==a].index),20)
    df_temp = tw.loc[rows]
    df_new=df_new.append(df_temp,ignore_index=True)    
    X_train, X_test, Y_train, Y_test = train_test_split(df_temp.loc[:,['Tweet']], df_temp.loc[:,['User']], test_size=0.2, random_state=42)
    twts_train=twts_train.append(X_train, verify_integrity=False)
    twts_test=twts_test.append(X_test, verify_integrity=False)
    author_train=author_train.append(Y_train, verify_integrity=False)
    author_test=author_test.append(Y_test, verify_integrity=False)
    
    

In [123]:
print (len(twts_train),len(author_train))

84896 84896


In [124]:
print(len(twts_test),len(author_test))

21224 21224


In [125]:
df_new.head()

,User,Tweet,num_of_words
0,8746,Going to watch Grey's on the big screen - Thur...,11
1,8746,RT REF RT REF REF fab white shirts are a must ...,17
2,8746,REF Without what? #anntaylor,4
3,8746,RT REF Retailers who aren’t engaging customers...,16
4,8746,RT REF Help us help MusiCares! Vote for Charit...,12


In [126]:
cnt_user = df_new['User'].value_counts()
cnt_user.describe()


count    5306.0
mean       20.0
std         0.0
min        20.0
25%        20.0
50%        20.0
75%        20.0
max        20.0
Name: User, dtype: float64

In [127]:
df_new.tail()

,User,Tweet,num_of_words
106115,5172,PSA regarding banana peels. URL - I pack them ...,16
106116,5172,"Small dinner, small bowl of cereal, NUM mile t...",13
106117,5172,URL - I found these rah rah USA bumper sticker...,20
106118,5172,REF tag implied by I crash SF sticker,8
106119,5172,WTF RT REF Sanjiv: ALL bicyclists are rude and...,21


In [128]:
tw.shape

(312494, 3)

In [129]:
df_new.shape

(106120, 3)

In [130]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
def text_process(text):
    """
    Takes in a string of text, then performs the following:
    1. Tokenizes and removes punctuation
    3. Stems
    4. Returns a list of the cleaned text
    """

    # tokenizing
    tokenizer = RegexpTokenizer(r'\w+')
    text_processed=tokenizer.tokenize(text)
    
    
    # steming
    porter_stemmer = PorterStemmer()
    
    text_processed = [porter_stemmer.stem(word) for word in text_processed]
    

    return text_processed

In [131]:
from sklearn.metrics import classification_report

In [132]:
ScoreSummaryByModel = list()
ScoreSummaryByModelParams=list()

In [133]:
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
def PredictionEvaluation(author_test_b,author_predicted_b,target_names,comment):
    Accuracy=accuracy_score(author_test_b,author_predicted_b)
    #print (Accuracy)
    Recall=recall_score(author_test_b, author_predicted_b, labels=[0,1,2,3], average='macro')
    #print (Recall)
    Precision=precision_score(author_test_b, author_predicted_b, labels=[0,1,2,3], average='macro')
    #print (Precision)
    F1=f1_score(author_test_b, author_predicted_b, labels=[0,1,2,3], average='macro')
    #print (F1)
    ScoreSummaryByModel.append([Accuracy,Recall,Precision,F1,comment])
    print(classification_report(author_test_b, author_predicted_b, target_names=target_names))

In [134]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.model_selection import learning_curve, GridSearchCV
from sklearn.preprocessing import LabelBinarizer

def ModelParamsEvaluation (f_union,model,params,comment):
    pipeline = Pipeline([
    # Extract the text & text_coded
    # Use FeatureUnion to combine the features from different vectorizers
    ('union', f_union),
    # Use a  classifier on the combined features
    ('clf', model)
    ])
    
    grid_search = GridSearchCV(estimator=pipeline, param_grid=params, verbose=1)
    grid_search.fit(twts_train['Tweet'], author_train['User'])
    author_predicted = grid_search.predict(twts_test['Tweet'])
    lb = LabelBinarizer()
    author_test_b = lb.fit_transform(author_test['User'])
    author_predicted_b  = lb.fit_transform(author_predicted)
    #best score
    print("Best score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    author_names=grid_search.best_estimator_.named_steps['clf'].classes_

    for param_name in sorted(params.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
    for param_name in sorted(params.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
        ScoreSummaryByModelParams.append([comment,grid_search.best_score_,"\t%s: %r" % (param_name, best_parameters[param_name])]) 
    return (author_predicted,author_predicted_b,author_test_b,author_names),grid_search

In [135]:
f2_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(2, 4))),
            ])),
            # Pipeline for pulling stememd word features from the text
            ('text', Pipeline([
                ('tfidf',    TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1))),
            ])),        

        ],

    )

In [ ]:
%%time
from sklearn.svm import LinearSVC
#LinearSVC
from sklearn.svm import LinearSVC
#LinearSVC
p = {'clf__C': (1,0.1,0.01,0.001,0.0001)}
(author_predicted,author_predicted_b, author_test_b,author_names),Grid=ModelParamsEvaluation(f2_union,LinearSVC(),p,'LinearSVC')

C:\Users\AALI\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [ ]:
Grid.best_score_

In [112]:
PredictionEvaluation(author_predicted_b, author_test_b,author_names,'LinearSVC')

              precision    recall  f1-score   support

        1017       0.10      0.10      0.10        10
        1071       0.90      0.60      0.72        15
        1113       1.00      0.77      0.87        13
        1171       0.70      0.58      0.64        12
        1197       0.40      0.40      0.40        10
        1255       1.00      0.71      0.83        14
        1317       0.30      0.38      0.33         8
        1350       0.20      0.22      0.21         9
        1353       0.90      0.56      0.69        16
        1382       0.20      0.33      0.25         6
        1416       0.60      0.43      0.50        14
        1463       0.80      0.50      0.62        16
        1509       0.20      0.40      0.27         5
        1516       0.10      0.09      0.10        11
        1518       0.10      0.14      0.12         7
        1523       0.50      0.45      0.48        11
        1541       0.30      0.25      0.27        12
        1590       1.00    

In [83]:
file1 = "data/test_tweets_unlabeled.txt"
temp=[]
with open(file1, 'r',encoding="utf-8") as data:
  temp = [line for line in data] 

test_unlabeled = pd.DataFrame(temp,columns = ["Tweet"])
test_unlabeled.head()
data=Grid.predict(test_unlabeled["Tweet"])
submission_file(data)

In [118]:
#unlabel_pred = Grid.predict(test_unlabeled["Tweet"])
User_predicted =Grid.predict(test_unlabeled["Tweet"])
lb = LabelBinarizer()
print(unlabel_pred)
#submission_file(unlabel_pred)

print(unlabel_pred)


['536' '3085' '6492' ... '4429' '6492' '6652']
['536' '3085' '6492' ... '4429' '6492' '6652']


In [80]:
print(len(unlabel_pred))
for count,predicted in enumerate(unlabel_pred):
          print([count+1,predicted])

35437
[1, '4267']
[2, '6428']
[3, '4974']
[4, '4562']
[5, '2203']
[6, '3989']
[7, '1698']
[8, '7362']
[9, '2586']
[10, '4974']
[11, '7362']
[12, '5519']
[13, '4974']
[14, '4974']
[15, '6428']
[16, '4974']
[17, '1896']
[18, '1669']
[19, '6841']
[20, '1698']
[21, '2586']
[22, '4267']
[23, '4267']
[24, '2110']
[25, '1171']
[26, '4185']
[27, '2110']
[28, '319']
[29, '2586']
[30, '1669']
[31, '3989']
[32, '3220']
[33, '4185']
[34, '1896']
[35, '2686']
[36, '4562']
[37, '4267']
[38, '5519']
[39, '6428']
[40, '4185']
[41, '319']
[42, '5519']
[43, '1669']
[44, '2686']
[45, '319']
[46, '2586']
[47, '6898']
[48, '5519']
[49, '2586']
[50, '5519']
[51, '4562']
[52, '3989']
[53, '3989']
[54, '4185']
[55, '2586']
[56, '6428']
[57, '4974']
[58, '4267']
[59, '4974']
[60, '319']
[61, '6898']
[62, '4974']
[63, '9911']
[64, '1171']
[65, '6428']
[66, '5519']
[67, '4562']
[68, '1669']
[69, '4974']
[70, '4925']
[71, '2686']
[72, '1698']
[73, '6898']
[74, '1698']
[75, '1669']
[76, '4974']
[77, '1669']
[78, '

[2442, '4185']
[2443, '2110']
[2444, '6428']
[2445, '2586']
[2446, '4562']
[2447, '4925']
[2448, '4974']
[2449, '3989']
[2450, '4562']
[2451, '1669']
[2452, '4974']
[2453, '7362']
[2454, '2203']
[2455, '7362']
[2456, '5519']
[2457, '319']
[2458, '4562']
[2459, '2110']
[2460, '1896']
[2461, '4974']
[2462, '4185']
[2463, '2686']
[2464, '4562']
[2465, '1669']
[2466, '4562']
[2467, '4974']
[2468, '3989']
[2469, '3220']
[2470, '2110']
[2471, '2110']
[2472, '6428']
[2473, '4974']
[2474, '319']
[2475, '5519']
[2476, '1698']
[2477, '4562']
[2478, '6428']
[2479, '1669']
[2480, '4974']
[2481, '2586']
[2482, '4562']
[2483, '3989']
[2484, '4974']
[2485, '4974']
[2486, '319']
[2487, '1171']
[2488, '5519']
[2489, '6428']
[2490, '4974']
[2491, '4974']
[2492, '4974']
[2493, '4185']
[2494, '6428']
[2495, '3989']
[2496, '2586']
[2497, '4974']
[2498, '1171']
[2499, '2586']
[2500, '4974']
[2501, '2203']
[2502, '3220']
[2503, '4974']
[2504, '4974']
[2505, '4974']
[2506, '7362']
[2507, '1669']
[2508, '2586'

[4771, '3989']
[4772, '4974']
[4773, '1171']
[4774, '319']
[4775, '4562']
[4776, '2586']
[4777, '4925']
[4778, '3989']
[4779, '6841']
[4780, '3220']
[4781, '4974']
[4782, '4925']
[4783, '4562']
[4784, '2203']
[4785, '3989']
[4786, '4562']
[4787, '1669']
[4788, '4185']
[4789, '4562']
[4790, '1171']
[4791, '319']
[4792, '4974']
[4793, '6428']
[4794, '4562']
[4795, '5519']
[4796, '6898']
[4797, '1669']
[4798, '4562']
[4799, '7362']
[4800, '2203']
[4801, '4974']
[4802, '1896']
[4803, '6898']
[4804, '1171']
[4805, '319']
[4806, '2110']
[4807, '4974']
[4808, '4562']
[4809, '1896']
[4810, '4974']
[4811, '1698']
[4812, '4562']
[4813, '4562']
[4814, '6428']
[4815, '1896']
[4816, '6428']
[4817, '4562']
[4818, '2110']
[4819, '2686']
[4820, '1698']
[4821, '4974']
[4822, '319']
[4823, '2586']
[4824, '4974']
[4825, '1698']
[4826, '1698']
[4827, '4562']
[4828, '6428']
[4829, '2110']
[4830, '1669']
[4831, '7362']
[4832, '319']
[4833, '2686']
[4834, '4974']
[4835, '4267']
[4836, '4974']
[4837, '6428']


[7441, '4974']
[7442, '4974']
[7443, '4974']
[7444, '3989']
[7445, '2686']
[7446, '7362']
[7447, '4562']
[7448, '4562']
[7449, '3989']
[7450, '4185']
[7451, '319']
[7452, '1171']
[7453, '1669']
[7454, '2586']
[7455, '6428']
[7456, '5519']
[7457, '6428']
[7458, '2203']
[7459, '3220']
[7460, '4267']
[7461, '4562']
[7462, '4925']
[7463, '4267']
[7464, '4185']
[7465, '2203']
[7466, '2686']
[7467, '4974']
[7468, '6898']
[7469, '1698']
[7470, '2110']
[7471, '1669']
[7472, '4974']
[7473, '4185']
[7474, '6428']
[7475, '4974']
[7476, '4974']
[7477, '3989']
[7478, '5519']
[7479, '4974']
[7480, '319']
[7481, '1669']
[7482, '1896']
[7483, '319']
[7484, '4974']
[7485, '319']
[7486, '319']
[7487, '1896']
[7488, '4974']
[7489, '4974']
[7490, '4267']
[7491, '1669']
[7492, '4267']
[7493, '6428']
[7494, '4185']
[7495, '2686']
[7496, '4974']
[7497, '4267']
[7498, '1698']
[7499, '4974']
[7500, '4267']
[7501, '1171']
[7502, '4185']
[7503, '2586']
[7504, '4562']
[7505, '5519']
[7506, '2586']
[7507, '319']
[

[10195, '4185']
[10196, '4185']
[10197, '319']
[10198, '1698']
[10199, '4974']
[10200, '2586']
[10201, '1698']
[10202, '6428']
[10203, '319']
[10204, '1698']
[10205, '6428']
[10206, '6428']
[10207, '4974']
[10208, '3220']
[10209, '3220']
[10210, '6898']
[10211, '2686']
[10212, '1669']
[10213, '4974']
[10214, '4974']
[10215, '2586']
[10216, '2686']
[10217, '7362']
[10218, '319']
[10219, '1669']
[10220, '4974']
[10221, '2110']
[10222, '6428']
[10223, '319']
[10224, '4562']
[10225, '2203']
[10226, '2203']
[10227, '4974']
[10228, '6841']
[10229, '6898']
[10230, '2203']
[10231, '3220']
[10232, '4974']
[10233, '4562']
[10234, '2110']
[10235, '1669']
[10236, '2586']
[10237, '3989']
[10238, '4974']
[10239, '3220']
[10240, '2203']
[10241, '3989']
[10242, '3220']
[10243, '1669']
[10244, '5519']
[10245, '4974']
[10246, '3989']
[10247, '6428']
[10248, '4562']
[10249, '2586']
[10250, '4974']
[10251, '4974']
[10252, '1669']
[10253, '1669']
[10254, '3220']
[10255, '1669']
[10256, '4974']
[10257, '551

...

[24937, '4974']
[24938, '319']
[24939, '4974']
[24940, '1669']
[24941, '1669']
[24942, '319']
[24943, '7362']
[24944, '2586']
[24945, '4974']
[24946, '1669']
[24947, '319']
[24948, '1669']
[24949, '6898']
[24950, '4267']
[24951, '4974']
[24952, '6428']
[24953, '2110']
[24954, '3989']
[24955, '4974']
[24956, '4974']
[24957, '1896']
[24958, '3220']
[24959, '4267']
[24960, '1171']
[24961, '3989']
[24962, '1698']
[24963, '2586']
[24964, '6428']
[24965, '4562']
[24966, '5519']
[24967, '4562']
[24968, '4974']
[24969, '3989']
[24970, '6841']
[24971, '2586']
[24972, '1669']
[24973, '5519']
[24974, '2586']
[24975, '6898']
[24976, '2586']
[24977, '4974']
[24978, '6428']
[24979, '6428']
[24980, '1669']
[24981, '4974']
[24982, '1698']
[24983, '4267']
[24984, '4562']
[24985, '4974']
[24986, '4974']
[24987, '4185']
[24988, '2586']
[24989, '1669']
[24990, '3220']
[24991, '3220']
[24992, '1896']
[24993, '4185']
[24994, '4974']
[24995, '4267']
[24996, '3220']
[24997, '7362']
[24998, '1171']
[24999, '49

[27934, '4974']
[27935, '2203']
[27936, '4562']
[27937, '2110']
[27938, '4974']
[27939, '319']
[27940, '4267']
[27941, '1698']
[27942, '6428']
[27943, '1669']
[27944, '6841']
[27945, '3989']
[27946, '5519']
[27947, '6428']
[27948, '4185']
[27949, '2110']
[27950, '1669']
[27951, '1698']
[27952, '3220']
[27953, '6428']
[27954, '3989']
[27955, '1171']
[27956, '6428']
[27957, '6428']
[27958, '6428']
[27959, '4562']
[27960, '5519']
[27961, '1896']
[27962, '4562']
[27963, '319']
[27964, '4185']
[27965, '4974']
[27966, '3220']
[27967, '9911']
[27968, '4974']
[27969, '4185']
[27970, '2586']
[27971, '2686']
[27972, '6841']
[27973, '2586']
[27974, '1171']
[27975, '4974']
[27976, '1669']
[27977, '4974']
[27978, '4974']
[27979, '6898']
[27980, '319']
[27981, '3989']
[27982, '4185']
[27983, '9911']
[27984, '5519']
[27985, '3989']
[27986, '4185']
[27987, '6898']
[27988, '6428']
[27989, '2586']
[27990, '4562']
[27991, '2110']
[27992, '9911']
[27993, '6898']
[27994, '6428']
[27995, '319']
[27996, '497

[30436, '319']
[30437, '6898']
[30438, '4185']
[30439, '4974']
[30440, '1698']
[30441, '2586']
[30442, '2686']
[30443, '4562']
[30444, '6428']
[30445, '2686']
[30446, '1896']
[30447, '6428']
[30448, '4974']
[30449, '9911']
[30450, '1669']
[30451, '4974']
[30452, '4562']
[30453, '4267']
[30454, '6428']
[30455, '1896']
[30456, '4974']
[30457, '1698']
[30458, '4974']
[30459, '2686']
[30460, '3989']
[30461, '1896']
[30462, '4267']
[30463, '4974']
[30464, '1669']
[30465, '1698']
[30466, '4267']
[30467, '1698']
[30468, '5519']
[30469, '2110']
[30470, '6898']
[30471, '4974']
[30472, '3220']
[30473, '2586']
[30474, '6428']
[30475, '6898']
[30476, '319']
[30477, '6428']
[30478, '2110']
[30479, '4562']
[30480, '4974']
[30481, '2586']
[30482, '4974']
[30483, '4562']
[30484, '4974']
[30485, '2110']
[30486, '4185']
[30487, '1669']
[30488, '6428']
[30489, '7362']
[30490, '6428']
[30491, '1669']
[30492, '4974']
[30493, '4974']
[30494, '6428']
[30495, '5519']
[30496, '4185']
[30497, '6841']
[30498, '4

[33436, '3220']
[33437, '1698']
[33438, '4267']
[33439, '3220']
[33440, '4974']
[33441, '1698']
[33442, '1171']
[33443, '1669']
[33444, '4974']
[33445, '4974']
[33446, '4267']
[33447, '6428']
[33448, '4185']
[33449, '4925']
[33450, '3220']
[33451, '4974']
[33452, '319']
[33453, '2586']
[33454, '3220']
[33455, '3989']
[33456, '1171']
[33457, '1669']
[33458, '6898']
[33459, '2586']
[33460, '4974']
[33461, '3989']
[33462, '3220']
[33463, '1698']
[33464, '2203']
[33465, '1171']
[33466, '3220']
[33467, '4974']
[33468, '7362']
[33469, '4974']
[33470, '319']
[33471, '4974']
[33472, '2110']
[33473, '1669']
[33474, '4974']
[33475, '4974']
[33476, '4974']
[33477, '4974']
[33478, '1896']
[33479, '1698']
[33480, '7362']
[33481, '4974']
[33482, '319']
[33483, '1171']
[33484, '1669']
[33485, '4562']
[33486, '2686']
[33487, '9911']
[33488, '4974']
[33489, '6841']
[33490, '1171']
[33491, '1698']
[33492, '2110']
[33493, '2110']
[33494, '2110']
[33495, '4267']
[33496, '4185']
[33497, '4267']
[33498, '31

[35436, '6428']
[35437, '5519']
